In [44]:
import argparse
from openai import AzureOpenAI
import openai
import os

current_directory = os.path.dirname(os.path.abspath("__file__"))
os.chdir(current_directory)

In [45]:
client = AzureOpenAI(
    api_version="2023-05-15",
    azure_endpoint=" ", # replace with azure endpoint
    api_key="", # replace with API key
)

In [46]:
def generate_prompt(question, result, prompt_file):
    with open(prompt_file, "r") as f:
        prompt = f.read()       

    prompt = prompt.format(
        user_question=question, result = result)
    
    return prompt

In [47]:
def Interpret_result(question, result, prompt_file):
    
    prompt = generate_prompt(question, result, prompt_file)
    try:
        sys_prompt = prompt.split("### Input:")[0]
        user_prompt = prompt.split("### Input:")[1].split("### Response:")[0]
        #assistant_prompt = prompt.split("### Response:")[1]
    except:
        raise ValueError("Invalid prompt file. Please use Interpret_result_prompt.md")
  
    messages = []
    messages.append({"role": "system", "content": sys_prompt})
    messages.append({"role": "user", "content": user_prompt})

    response = client.chat.completions.create(
        model="chat",
        messages=messages,
        temperature = 0.5
    )

    return response.model_dump()["choices"][0]["message"]["content"]

In [49]:
result_file= '..GPT_pipeline/postgreSQL_database_assessment/benchmarking_postgres_answers.txt'
prompt_file = '../GPT_pipeline/postgreSQL_database_assessment/Interpret_result/Interpret_result_prompt.md'

# save a question and its result from one of the benchmark questions asked to the database
with open(result_file, 'r') as f:
    result_string = f.read()
    rows = result_string.split('\n')
    
    
    for i,row in enumerate(rows): 
        question = row.split('\t')[0]
        # result is the second element of the row
        result = row.split('\t')[2].replace('(','').replace(')','')
        interpretation = Interpret_result(question, result, prompt_file)
        with open('./interpreted_results.txt', 'a') as file:
             file.write(f"{i}\t{interpretation.strip()}\n\n")
                
                


In [21]:
prompt = generate_prompt(question, result, prompt_file)

In [22]:
prompt


"### Instructions:\nYour task is to contextualize an answer regarding human microbiome and its numerical result with state of art knowledge.\n\n\n### Input:\nGiven this question: '2 What is the distribution(average level) of faecalibacterium prausnitzii in the database?' asked to a human microbiome database returning result 4.749735057239966,, provide comment. \n\n\n\n### Response:\nGiven the question 2 What is the distribution(average level) of faecalibacterium prausnitzii in the database?, 4.749735057239966, is..."